! pip install opencv-python==4.5.5.64

In [1]:
import cv2
import os
import numpy as np
import mediapipe as mp
import pandas as pd
import matplotlib.pyplot as plt
from detector import get_info

Using cache found in C:\Users\Virginia Riego/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-1-24 Python-3.7.4 torch-1.9.0+cu102 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [2]:
def process_image(user_id,img_name):
    info = img_name.split('_')
    info_hand = info[2]
    user_sex = info[1]
    frame = info[3].split('.')[0]
    sign = info_hand[-1]
    hand =  'Right' if info_hand[0]== 'r' else 'Left'
    distance =  'Near' if info_hand[1]== 'n' else 'Far'
    
    image = cv2.imread(os.path.join('..','images',user_id,img_name))
    image = cv2.flip(image, 1) # Mirror image
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    try:
        users = get_info(image)
    except Exception as e:
        print('Error', img_name, e)
        return None
    if len(users)==1 and users[0][hand] is not None:
        hand_points = users[0][hand]
        final = [user_id, user_sex, sign, frame, hand, distance]
        final.extend(hand_points)
        return final
    else:
        return None

In [3]:
columns = ['user', 'sex', 'sign','frame', 'hand', 'distance']
marks_names = ['hand_mark_'+str(i).zfill(2)+axis for i in range(21) for axis in ['x','y','z']]
columns.extend(marks_names)

f_name= 'results/hand_landmarks.csv'

In [4]:
df_users = pd.read_csv('users.csv')
df_users.head()

,name_folder,user_id,sex,train
0,adrian,user01,0,1
1,alexis,user02,0,1
2,bea,user03,1,1
3,camino,user04,1,1
4,claudia,user05,1,1


In [5]:
not_detected = []
df = pd.DataFrame(columns=columns)
df.to_csv(f_name, index=False, mode='w')
for index, row in df_users.iterrows():
    name_folder = row.name_folder
    user_id = row.user_id
    user_identifier = int(user_id[4:])
    sex = row.sex   
        
    images = [i for i in os.listdir(os.path.join('..','images',user_id)) if i.endswith('.png')]
    cont = 1
    
    print(user_id, len(images))

    for i in images:
        info = process_image(user_id,i)
        if info is None:
            not_detected.append(i)
            #print('\t-', i)
        else:
            df = pd.DataFrame(dict(zip(columns,info)), index=[cont])
            df.to_csv(f_name, index=False, header=False, mode='a')
            #print('\t+', i)
        cont+=1
with open('results/not_hand_detected.txt', 'w') as fp:
    for item in not_detected:
        # write each item on a new line
        fp.write("%s\n" % item)

user01 580
user02 608
user03 817
user04 533
user05 610
user06 719
user07 563
user08 746
user09 381
user10 908
user11 387
user12 622
user13 675
user14 626
